In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from dataset_loader import *
from utils import *
from datasets import Dataset, DatasetDict
import datasets
import logging
import warnings
logging.disable(logging.WARNING) 
datasets.set_progress_bar_enabled(False)
warnings.simplefilter("ignore")

SEED = 84
torch.manual_seed(SEED)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [14]:
dataset_name = 'sentiment'
ds = get_ds(dataset_name)

In [15]:
model_name =  'huawei-noah/TinyBERT_General_4L_312D'
folder_name = 'tinybert/new'

In [27]:
set_seed()
model = load_model(model_name)
tokenized_data = tokenize_dataset(ds, tokenizer_name=model_name, max_length = 64)
_ = train(model, tokenized_data, path = f'{folder_name}/{dataset_name}', num_train_epochs=10)
#model.save_pretrained(f'{folder_name}/{dataset_name}/model')

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.431738,0.812627
2,No log,0.368633,0.852342
3,No log,0.293868,0.883910
4,No log,0.322088,0.890020
5,0.284000,0.336895,0.891039
6,0.284000,0.424200,0.892057
7,0.284000,0.401126,0.889002
8,0.284000,0.424535,0.893075
9,0.284000,0.386894,0.901222
10,0.095300,0.382330,0.892057


### save best model's folder as 'model'

In [29]:
model = load_model(f'{folder_name}/{dataset_name}/model').to(device).eval()
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [30]:
tokenized_data = tokenize_dataset(ds, tokenizer_name=model_name, max_length = 64)
train(model, tokenized_data, path=f'{folder_name}/{dataset_name}', evaluate = True)['eval_accuracy']

0.9012219959266803

In [31]:
encoded = tokenizer.encode("i love movies", add_special_tokens=True, return_tensors="pt").to(device)
traced_model = torch.jit.trace(model, encoded)

In [32]:
torch.jit.save(traced_model, f"{folder_name}/{dataset_name}/traced.pt")
traced_model = torch.jit.load(f"{folder_name}/{dataset_name}/traced.pt")

In [33]:
predicting_sentences(model, tokenizer, device, [tokenizer.tokenize("i hate movies")])

tensor([[ 2.6348, -2.5838]], device='cuda:0', grad_fn=<AddmmBackward0>)


array([0])

In [34]:
predicting_sentences(model, tokenizer, device, [tokenizer.tokenize("i love movies")])

tensor([[-2.5861,  2.5458]], device='cuda:0', grad_fn=<AddmmBackward0>)


array([1])

In [13]:
per_class_accuracy(folder_name, model_name, 'corona')

positive accuracy 0.8960138648180243


negative_accuracy 0.9149191132414619


In [18]:
per_class_accuracy(folder_name, model_name, 'sentiment')

positive accuracy 0.9130434782608695
negative_accuracy 0.9261418853255587


In [19]:
per_class_accuracy(folder_name, model_name, 'dilemma')

positive accuracy 0.9247706422018349
negative_accuracy 0.9392789373814042
